In [ ]:
import requests
import json
import pandas as pd
import time

# Fetch flight status from OpenSky Network API
def fetch_flight_status(icao24, begin, end):
    """
    Fetch flight status using OpenSky API.

    Parameters:
    icao24 (str): Unique ICAO 24-bit address of the aircraft.
    begin (int): Start time in UNIX timestamp.
    end (int): End time in UNIX timestamp.

    Returns:
    dict: Flight status information.
    """
    url = f"https://opensky-network.org/api/flights/aircraft?icao24={icao24}&begin={begin}&end={end}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching flight status: {e}")
        return None

# Save flight data to a local file
def save_flight_data(data, filename):
    try:
        df = pd.DataFrame(data)
        df.to_csv(filename, index=False)
        print(f"Data saved to {filename}")
    except Exception as e:
        print(f"Error saving data to {filename}: {e}")

# Convert human-readable time to UNIX timestamp
def convert_to_unix_time(date_str):
    """
    Convert a human-readable date to UNIX timestamp.

    Parameters:
    date_str (str): Date in 'YYYY-MM-DD HH:MM:SS' format.

    Returns:
    int: UNIX timestamp.
    """
    try:
        pattern = "%Y-%m-%d %H:%M:%S"
        return int(time.mktime(time.strptime(date_str, pattern)))
    except ValueError as e:
        print(f"Error parsing date: {e}")
        return None

# Main function to fetch and save flight status
def main():
    print("Welcome to the Flight Status Tracker!")
    icao24 = input("Enter the ICAO24 code of the aircraft (e.g., abc123): ")
    begin_str = input("Enter the start time (YYYY-MM-DD HH:MM:SS): ")
    end_str = input("Enter the end time (YYYY-MM-DD HH:MM:SS): ")
    filename = input("Enter the filename to save the data (e.g., flight_data.csv): ")

    begin = convert_to_unix_time(begin_str)
    end = convert_to_unix_time(end_str)

    if begin is None or end is None:
        print("Invalid date format. Please use 'YYYY-MM-DD HH:MM:SS'.")
        return

    print("Fetching flight status...")
    flight_data = fetch_flight_status(icao24, begin, end)

    if flight_data:
        save_flight_data(flight_data, filename)
    else:
        print("No data retrieved or an error occurred.")

if __name__ == "__main__":
    main()
